In [1]:
import pandas as pd

from average_speed_calculator import AverageSpeedCalculator
from power_consumption_calculator import PowerConsumptionCalculator
from motor_rating_calculator import MotorRatingCalculator
from csv_loader import CSVLoader  # Import the new CSVLoader class

class TrainSimulation:
    def __init__(self, 
                 train_params_dict:dict, 
                 stations:pd.core.frame.DataFrame, 
                 curves:pd.core.frame.DataFrame,
                 gradients:pd.core.frame.DataFrame, 
                 speed_restrictions:pd.core.frame.DataFrame,):

        self.train_params_dict = train_params_dict
        self.stations = stations
        self.curves = curves
        self.gradients = gradients
        self.speed_restrictions = speed_restrictions

        self.time = 0
        self.speed = 0
        self.time_log = []
        self.speed_log = []

        self.average_speed_calculator = AverageSpeedCalculator()
        self.power_calculator = PowerConsumptionCalculator(
            float(train_params_dict['Train_mass']), 
            float(train_params_dict['Regeneration_efficiency'])
        )
        self.motor_rating_calculator = MotorRatingCalculator(self.power_calculator, self.average_speed_calculator)
        self.derived_values()

    # Derive the values from the passed arrays
    def derived_values(self):
        # Check coasting allowed
        self.no_coasting = False if train_params_dict['Coasting'] else True # Whether 'Coast' or not

        # Convert speeds from km/h to m/s
        self.max_speed = float(train_params_dict['Maximum_speed'])  * 1000 / 3600
        self.switch_speed = float(train_params_dict['Switch_speed'])  * 1000 / 3600

        # Extract the chainage and station names into list
        self.station_chainages = stations['Chainage'].values
        self.station_names = stations['Station_Name'].values

        # Distance calculations for controlling different run phase
        self.accelerating_distance = self.max_speed**2/2/float(train_params_dict['Acceleration_switch'])
        self.braking_distance = self.max_speed**2/2/float(train_params_dict['Braking_deceleration'])
        return 
    
    # Davis formula for resistance
    def davis(self, speed):
        static_friction = float(train_params_dict['Static_friction'])
        rolling_resistance = float(train_params_dict['Rolling_resistance'])
        air_resistance = float(train_params_dict['Air_resistance'])
        train_mass = float(train_params_dict['Train_mass'])
        return (static_friction + rolling_resistance * speed + air_resistance * speed**2) * train_mass

    # Helper function to log time and speed
    def log_simulation_data(self, time, speed):
        self.time_log.append(time / 60)  # Convert time to minutes for logging
        self.speed_log.append(speed * 18 / 5)  # Convert m/s to km/h for logging   
        
    def print_values(self):
        self.log_simulation_data(1,2)
        return self.speed_log

    
    def run_simulation(self):
        # Implement the simulation logic here using the imported classes
        pass

if __name__ == "__main__":
    # Initialize the CSV loader
    csv_loader = CSVLoader()

    # List of CSV files and their labels
    csv_files = [
        ('train_parameters.csv', 'train_params'),
        ('stations.csv', 'stations'),
        ('curves.csv', 'curves'),
        ('gradients.csv', 'gradients'),
        ('sr.csv', 'speed_restrictions')
    ]
    
    # Load data from CSV files
    data_dict = csv_loader.load_csv_list(csv_files)

    # Access the loaded data
    train_params_dict = data_dict['train_params']
    stations = data_dict['stations']
    curves = data_dict['curves']
    gradients = data_dict['gradients']
    speed_restrictions = data_dict['speed_restrictions']

    # Initialize the TrainSimulation class with parameters from the CSV
    simulation = TrainSimulation(train_params_dict, stations, curves, gradients, speed_restrictions)
    
    print(simulation.print_values())

    # Run the simulation
    simulation.run_simulation()


[7.2]


In [2]:
print(float(train_params_dict['Acceleration']))

1.0
